<a href="https://colab.research.google.com/github/Vrinda999/sql-command-gen/blob/main/sql_command_gen.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install torch transformers bitsandbytes accelerate sqlparse

  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl.metadata (1.8 kB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-manylinu

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM

In [ ]:
torch.cuda.is_available()

True

In [ ]:
available_memory = torch.cuda.get_device_properties(0).total_memory

In [ ]:
print(available_memory)

15835660288


### Downloading the Model

In [ ]:
model_name = "defog/sqlcoder-7b-2"
tokenizer = AutoTokenizer.from_pretrained(model_name)
if available_memory > 16e9:
    model = AutoModelForCausalLM.from_pretrained(
        model_name,
        trust_remote_code = True,
        torch_dtype = torch.float16,
        device_map = "auto",
        use_cache = True,
    )

else:
    model = AutoModelForCausalLM.from_pretrained(
        model_name,
        trust_remote_code = True,
        # torch_dtype = torch.float16,
        load_in_8bit = True,
        device_map = "auto",
        use_cache = True,
    )

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/1.84k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/515 [00:00<?, ?B/s]

...

model-00001-of-00003.safetensors:   0%|          | 0.00/4.94G [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/4.95G [00:00<?, ?B/s]

model-00003-of-00003.safetensors:   0%|          | 0.00/3.59G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/111 [00:00<?, ?B/s]

### Setting the Question & Prompt, and Tokenize.

In [ ]:
prompt = """### Task
Generate an SQL Query to Answer [QUESTION]{question}[/QUESTION]

### Instructions
- If you cannot answer the qustion with the available database schema, return "I don't Know".
- Remember that revnue is Price multiplied by Quantity.
- Remember that Cost is supply_price multiplied by Quantity.

### Database Schema
This queru will run on a database whose schema is represented in this string:
CREATE TABLE products (
    product_id INTEGER PRIMARY KEY, -- unique ID for each product
    Name VARCHAR(50), -- Name of the Product
    Price DECIMAL(10,2), -- Price of each unit of Product
    Quantity INTEGER -- Current Quantity in stock
);

CREATE TABLE Customers (
    customer_id INTEGER PRIMARY KEY, -- Unique ID for each customer
    Name VARCHAR(50), -- Name of the Customer
    address VARCHAR(100) -- Mailing Address of the Customer
);

CREATE TABLE Salespeople (
    salesperson_id INTEGER PRIMARY KEY, -- Unique ID for each Salesperson
    Name VARCHAR(50), -- Name of the Salesperson
    Region VARCHAR(50) -- Geographic Sales Region
);

CREATE TABLE sales (
    sale_id INTEGER PRIMARY KEY, -- Unique ID for each Sale
    product_id INTEGER, -- ID of the Product Sold
    customer_id INTEGER, -- ID of the Customer who made the Purchase
    salesperson_id INTEGER, -- ID of th Salesperson who made the Sale
    sale_date DATE, -- Date the Sale Occurred on
    Quantity INTEGER, -- Quantity of the Product Sold
);

CREATE TABLE product_suppliers (
    supplier_id INTEGER PRIMARY KEY, -- Unique ID for each Supplier
    product_id INTEGER, -- Product ID supplied
    supply_price DECIMAL(10,2) -- Unit Price Charged by Supplier
);

-- sales.product_id can be joined wih products.product_id
-- sales.customer_id can be joined with customers.customer_id
-- sales.salesperson_id can be joined with salespeople.salesperson_id
-- product_suppliers.product_id can be joined with products.product_id

### Answer
Given the Database Schema, here is the SQL Query that Answers [QUESTION]{question}[/QUESTION]
[SQL]
"""

### Generating SQL

In [ ]:
import sqlparse

def generate_query(question):
    updated_prompt = prompt.format(question = question)
    inputs = tokenizer(updated_prompt, return_tensors = "pt").to("cuda")
    generated_ids = model.generate(
        **inputs,
        num_return_sequences = 1,
        eos_token_id = tokenizer.eos_token_id,
        pad_token_id = tokenizer.eos_token_id,
        max_new_tokens = 400,
        do_sample = False,
        num_beams = 1,
    )
    outputs = tokenizer.batch_decode(generated_ids, skip_special_tokens = True)

    torch.cuda.empty_cache()
    torch.cuda.synchronize()

    return sqlparse.format(outputs[0].split("[SQL]")[-1], reindent = True)

In [ ]:
question = "What was the Highest Quantity Sold Last Month?"
generated_sql = generate_query(question)
print(generated_sql)


SELECT MAX(s.Quantity) AS max_quantity
FROM sales s
WHERE s.sale_date >= (CURRENT_DATE - INTERVAL '1 month');


In [ ]:
question = "Which salesperson sold Large Amount of Products Last Month"
generated_sql = generate_query(question)
print(generated_sql)


SELECT s.salesperson_id,
       SUM(s.quantity) AS total_quantity
FROM sales s
WHERE s.sale_date >= (CURRENT_DATE - INTERVAL '1 month')
GROUP BY s.salesperson_id
ORDER BY total_quantity DESC NULLS LAST
LIMIT 1;


In [ ]:
question = "What was our Revenue by Product in the New York Region Last Month?"
generated_sql = generate_query(question)
print(generated_sql)


SELECT p.product_id,
       SUM(s.Quantity * p.Price) AS revenue
FROM sales s
JOIN salespeople sp ON s.salesperson_id = sp.salesperson_id
JOIN products p ON s.product_id = p.product_id
WHERE sp.Region = 'New York'
  AND s.sale_date >= (CURRENT_DATE - INTERVAL '1 month')
GROUP BY p.product_id
ORDER BY revenue DESC NULLS LAST;
